In [1]:
!pip install bokeh
!pip install geopy
!pip install beautifulsoup4
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import requests
import json
from bs4 import BeautifulSoup

from bokeh.models import *
from bokeh.plotting import *
from bokeh.io import *
from bokeh.tile_providers import *
from bokeh.palettes import *
from bokeh.transform import *
from bokeh.layouts import *

You should consider upgrading via the 'c:\users\robbie\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\robbie\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\robbie\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


In [57]:

geolocator = Nominatim(user_agent="teproject")
_latitudes = []
_longitudes = []
_names = []
_industries = []
_startups = []

i = 2500
while i < 3500:
    base_overview_url = "https://api.builtin.com/companies/" + str(i) + "/overview"
    page = requests.get(base_overview_url)
    x = json.loads(page.content)
    try:
        x["alias"]
    except:
        i+=1
        continue
    alias = (x["alias"])
    alias = alias.split('/')[-1]
    industries = x["industries"]
    if(industries != None):
        industry = industries[0]
    name = x["title"]
    if alias == '':
        i+=1
        continue
    base_address_url = "https://api.builtin.com/companies/alias/" + alias + "?region_id=" + str(x["region_id"])
    page = requests.get(base_address_url)
    y = json.loads(page.content)
    try:
        y["total_employees"]
    except:
        i+=1
        continue
    if(y["total_employees"] == 0):
        i+=1
        continue
    if (y["total_employees"]) > 100:
        startup = False
    else:
        startup = True
    if(y["street_address_1"] == ""):
        i+=1
        continue
    address = y["street_address_1"] + " " + y["city"]
    try:
        location = geolocator.geocode(address)
    except:
        i+=1
        continue
    if location == None:
        i+=1
        continue
    latitude = location.latitude
    longitude = location.longitude
    _names.append(name)
    _industries.append(industry)
    _latitudes.append(latitude)
    _longitudes.append(longitude)
    _startups.append(startup)
    i+=1



In [58]:
#import and clean the data

su_df = pd.DataFrame({
    "name" : _names,
    "latitude": _latitudes,
    "longitude": _longitudes,
    "industry": _industries,
    "startup": _startups
})

su_df['latitude']=su_df['latitude'].astype('float')
su_df['longitude']=su_df['longitude'].astype('float')


In [59]:
#get a sense of what the data looks like 

su_df

,name,latitude,longitude,industry,startup
0,Greystone Technology,39.689908,-104.942328,Other,True
1,Fluid Market (Do Not Use/ Unpublished),39.752073,-104.986321,Mobile,True
2,Swimlane,39.962913,-105.169422,Security,True
3,Guerrero Howe Custom Media,41.896471,-87.635719,Agency,True
4,Tempesta Media,41.903670,-87.631023,Digital Media,True
...,...,...,...,...,...
360,Schomp Automotive,39.560055,-105.002161,Automotive,False
361,Aquilon Energy Services,41.807979,-88.072000,Fintech,True
362,The Future Project,40.735602,-73.991623,Edtech,True
363,Frontier Fire Protection,39.735804,-105.019778,Other,False


In [60]:
def wgs84_to_web_mercator(df, lon, lat):
    """Converts decimal longitude/latitude to Web Mercator format"""
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

df=wgs84_to_web_mercator(su_df,'longitude','latitude')

#Establishing a zoom scale for the map. The scale variable will also determine proportions for hexbins and bubble maps so that everything looks visually appealing. 

scale=50
x=df['x']
y=df['y']

#The range for the map extents is derived from the lat/lon fields. This way the map is automatically centered on the plot elements.

x_min=int(x.mean() - (scale * 350))
x_max=int(x.mean() + (scale * 350))
y_min=int(y.mean() - (scale * 350))
y_max=int(y.mean() + (scale * 350))

#Defining the map tiles to use. I use OSM, but you can also use ESRI images or google street maps.

tile_provider=get_provider(OSM)

#Establish the bokeh plot object and add the map tile as an underlay. Hide x and y axis.

plot=figure(
    title='Industry Distribution in Chicago 2020',
    match_aspect=True,
    tools='wheel_zoom,pan,reset,save',
    x_range=(x_min, x_max),
    y_range=(y_min, y_max),
    x_axis_type='mercator',
    y_axis_type='mercator',
    width=500
    )

plot.grid.visible=True

map=plot.add_tile(tile_provider)
map.level='underlay'

plot.xaxis.visible = False
plot.yaxis.visible=False
plot.title.text_font_size="20px"

output_notebook()

Loading BokehJS ...

In [74]:
#Creating a map that will display events categorically by industry
p=figure(
    title='2020 Chicago Tech Companies by Industry',
    match_aspect=True,
    tools='wheel_zoom,pan,reset,save',
    x_range=(x_min, x_max),
    y_range=(y_min, y_max),
    x_axis_type='mercator',
    y_axis_type='mercator',
    width=1000)

p.grid.visible=True

m=p.add_tile(tile_provider)
m.level='underlay'

p.xaxis.visible = False
p.yaxis.visible=False
p.title.text_font_size="20px"

#Creating a map that will display events categorically by startup
p1=figure(
    title='2020 Chicago Tech Companies by Startup',
    match_aspect=True,
    tools='wheel_zoom,pan,reset,save',
    x_range=(x_min, x_max),
    y_range=(y_min, y_max),
    x_axis_type='mercator',
    y_axis_type='mercator',
    width=1000)

p1.grid.visible=True

m=p1.add_tile(tile_provider)
m.level='underlay'

p1.xaxis.visible = False
p1.yaxis.visible=False
p1.title.text_font_size="20px"



In [75]:
events=list(su_df['industry'].dropna(how=None))
event_dict=dict.fromkeys(events,0)
events=list(event_dict)
events = ['AdTech', 'Healthtech',  'Consumer Web', 'Internet of Things', 'eCommerce', 'Edtech', 'Digital Media', 'Big Data', 'Cloud',  'Enterprise Web', 'Information Technology', 'Agency', 'Fintech', 'Other', 'Software', 'Artificial Intelligence', 'Food',  'HR Tech', 'Real Estate', 'Logistics', 'Marketing Tech', 'Professional Services']

startup_events = ['Startup', 'Non-Startup']
#Initializing empty lists to create temporary dataframes for each category

Startup=[]
NonStartup =[]

AdTech=[]
Healthtech=[]
Music=[]
ConsumerWeb =[]
InternetOfThings=[]
Edtech=[]
MachineLearning =[]
DigitalMedia=[]
SocialImpact=[]
EnterpriseWeb=[]
AngelorVCFirm=[]
Other=[]
Travel=[]
Events=[]
Fashion=[]
Payments=[]
Food=[]
Sales=[]
HRTech=[]
Gaming=[]
Insurance=[]
RealEstate=[]
Blockchain=[]
MarketingTech=[]
Mobile=[]
Cloud=[]
eCommerce=[]
Big_Data = []
Logistics = []
Fintech = []
Agency = []
Professional_Services = []
IT = []
Software = []
AI = []

startupList_list=[Startup,NonStartup]

list_list=[AdTech,
Healthtech,
ConsumerWeb,
InternetOfThings,
eCommerce,
Edtech,
DigitalMedia,
Big_Data,
Cloud,
EnterpriseWeb,
IT,
Agency,
Fintech,
Other,
Software,
AI,
HRTech,
RealEstate,
Logistics,
MarketingTech,
Professional_Services]

#Extracting event information for each category. I opted not to use nested iterators here to enhance code readability


for i in range(len(df['startup'])):
  if df.loc[i,'startup']==True:
    Startup.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  else:
    NonStartup.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])



for i in range(len(df['industry'])):
  if df.loc[i,'industry']=='AdTech':
    AdTech.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  if df.loc[i,'industry']=='Healthtech':
    Healthtech.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  if df.loc[i,'industry']=='Consumer Web':
    ConsumerWeb.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  if df.loc[i,'industry']=='Internet of Things':
    InternetOfThings.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  if df.loc[i,'industry']=='Edtech':
    Edtech.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  if df.loc[i,'industry']=='Digital Media':
    DigitalMedia.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  if df.loc[i,'industry']=='Enterprise Web':
    EnterpriseWeb.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  if df.loc[i,'industry']=='Cloud':
    Cloud.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  if df.loc[i,'industry']=='Other':
    Other.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  if df.loc[i,'industry']=='Food':
    Food.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  if df.loc[i,'industry']=='HR Tech':
    HRTech.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  if df.loc[i,'industry']=='Real Estate':
    RealEstate.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  if df.loc[i,'industry']=='Marketing Tech':
    MarketingTech.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  if df.loc[i,'industry']=='Professional Services':
    Professional_Services.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  if df.loc[i,'industry']=='Cloud':
    Cloud.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  if df.loc[i,'industry']=='eCommerce':
    eCommerce.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  if df.loc[i,'industry']=='Big_Data':
    Big_Data.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  if df.loc[i,'industry']=='Logistics':
    Logistics.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  if df.loc[i,'industry']=='Fintech':
    Fintech.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  if df.loc[i,'industry']=='Agency':
    Agency.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  if df.loc[i,'industry']=='Professional Services':
    Professional_Services.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  if df.loc[i,'industry']=='Information Technology':
    IT.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  if df.loc[i,'industry']=='Software':
    Software.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
  if df.loc[i,'industry']=='Artificial Intelligence':
    AI.append([ df.loc[i, 'name'],
                     df.loc[i,'industry'],
                     df.loc[i,'x'],df.loc[i,'y'],
                     df.loc[i,'startup'],
                     df.loc[i,'latitude'],
                     df.loc[i,'longitude']])
              
for i in range(len(list_list)):
  if(len(list_list[i]) == 1):
    print(events[i])

In [78]:
#using the list of lists, create temporary dataframes for each event category and plot them to our second map.

for i in range(len(list_list)):
  temp_df=pd.DataFrame(list_list[i],columns=['name', 'industry','x', 'y','startup','latitude','longitude'])
  temp_df.head()
  source=ColumnDataSource(temp_df)
  circle=p.circle(x='x',y='y',source=source,color=Turbo256[i*7],line_color=Turbo256[i*7],legend_label=events[i],hover_color='white',radius=100,fill_alpha=0.4)

  event_hover = HoverTool(tooltips=[('Name', '@name'),
                                   ('Industry','@industry'),
                                   ('Startup?', '@startup'),
                                   ('(Lat,Lon)','(@latitude,@longitude)')],
                          mode='mouse',
                          point_policy='follow_mouse',
                          renderers=[circle])
  
  event_hover.renderers.append(circle)
  p.tools.append(event_hover)


for i in range(len(startupList_list)):
  temp_df=pd.DataFrame(startupList_list[i],columns=['name', 'industry','x', 'y','startup','latitude','longitude'])
  temp_df.head()
  source=ColumnDataSource(temp_df)
  circle=p1.circle(x='x',y='y',source=source,color=Viridis3[i],line_color=Viridis3[i],legend_label=startup_events[i],hover_color='white',radius=100,fill_alpha=0.4)

  event_hover = HoverTool(tooltips=[('Name', '@name'),
                                   ('Industry','@industry'),
                                   ('Startup?', '@startup'),
                                   ('(Lat,Lon)','(@latitude,@longitude)')],
                          mode='mouse',
                          point_policy='follow_mouse',
                          renderers=[circle])
  
  event_hover.renderers.append(circle)
  p1.tools.append(event_hover)

#View our maps

p.legend.location = "top_right"
p.legend.click_policy="hide"
p1.legend.location = "top_right"
p1.legend.click_policy="hide"

show(row(p,p1))